# 비선형 분류 모형 종합 비교 (결측 80% 초과 변수 제거)

로지스틱, 경사하강법, KNN, SVM, 랜덤포레스트, XGBoost, LightGBM, CatBoost 결과를 불러와 **주요 성능 비교**, **과적합 여부**, **채택 모형 SHAP**을 한눈에 봅니다.

**사전 조건:** 각 `new_*.ipynb`를 실행해 `results/*.pkl`이 생성되어 있어야 합니다.

## 라이브러리 및 결과 로드

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sb
import shap

my_dpi = 100

results_dir = 'results'
pkl_names = [
    'new_로지스틱', 'new_경사하강법', 'new_KNN', 'new_SVM', 'new_랜덤포레스트',
    'new_XGBoost', 'new_LightGBM', 'new_CatBoost'
]

results = {}
for name in pkl_names:
    path = os.path.join(results_dir, name + '.pkl')
    if os.path.isfile(path):
        with open(path, 'rb') as f:
            results[name] = pickle.load(f)
        print(f'Loaded: {name}')
    else:
        print(f'Skip (not found): {path}')

if not results:
    raise SystemExit('결과 파일이 없습니다. 각 new_*.ipynb를 먼저 실행하세요.')

## 1. 주요 성능 비교표

In [ ]:
score_dfs = {k: v.get('score_df') for k, v in results.items() if v.get('score_df') is not None}
if score_dfs:
    summary_scores = pd.concat(score_dfs, axis=0)
    display(summary_scores)
else:
    print('score_df 없음')

## 2. 성능 비교 막대 그래프 (AUC 기준)

In [ ]:
if score_dfs:
    names = list(results.keys())
    aucs = []
    for n in names:
        df = results[n].get('score_df')
        if df is not None and not df.empty:
            a = df.get('AUC', df.get('roc_auc', pd.Series([None]))).iloc[0]
            aucs.append(a)
        else:
            aucs.append(None)
    fig, ax = plt.subplots(figsize=(10, 5), dpi=my_dpi)
    ax.barh(names, aucs)
    ax.set_xlabel('AUC')
    ax.set_title('모델별 AUC (결측 80% 초과 제거)')
    plt.tight_layout()
    plt.show()
else:
    print('score_df 없음')

## 3. 과적합 여부 요약

In [ ]:
for name, data in results.items():
    status = data.get('overfit_status', 'N/A')
    print(f'{name}: {status}')